### 1. 데이터분석의 목적
##### 타이타닉 데이터를 기반으로 생존자 분류예측

### 2. 프로젝트 계획 및 기간 설정
##### 1) 데이터의 품질을 확인하고 결측치 정리(train data/test data 모두)
##### 2) 피처들을 EDA해보면서(통계치 산출) 생존에 대한 의미 이해
##### 3) 최종적으로 모델에 활용한 피쳐를 선택하고 변환이 필요한 피쳐는 변환 및 추가
##### 4) 다양한 모델을 선택하여 모델별 예측값 도출(학습 및 결과 확인) 및 정확도 향상을 위한 다양한 시도
##### 5) 최종 모델을 통해 결과 kaggle에 제출
##### 6) 1~3번 과정 :2일 / 4~5번 과정 : 1일

### 3. 분석에 필요한 라이브러리,데이터 불러오기/데이터 품질 확인/결측치정리

In [51]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [52]:
# train / test data 불러오기
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [53]:
# 속성 설명
# PassengerId : 승객 아이디
# Survive : 생존여부
# Pclass : 티켓 등급(1 = 1st, 2 = 2nd, 3 = 3rd)
# Name : 승객이름
# Sex : 승객 성별
# Age : 승객 나이
# SibSp : 배에 탑승한 형제/자매/배우자
# Parch : 배에 탑승한 부모/자녀
# Ticket : 티켓 번호
# Fare : 티켓 요금
# Cabin : 객실번호
# Embarked : 승선항(C = Cherbourg, Q = Queenstown, S = Southampton)
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [59]:
# train data 데이터량 확인
# age / cabin 데이터가 전체 891 수량보다 적은걸로 봐서 결측치가 있는것으로 보임
# Embarked는 2개가 결측치로 보임
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [60]:
# isnull함수를 통해 정확한 결측치 수량 확인
# Age는 단순하게 채우기 보단 근거를 가지고 채울 수 있을것으로 판단됨
# Cabin 객실번호 피쳐는 빈값이 상당히 많기도 하고 결측치를 채울만한 근거가 부족할뿐만아니라 학습의 피쳐로써 직관적으로 의미가 부족해보이므로 제거 예정
# 승선항의 2개 결측치는 임의로 채워도 예측에 크게 문제가 되지 않을것 같다고 생각이 되며 티켓의 이름이나 운임요금 혹은 탑승자의 이름으로 유추하여 채울예정
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [56]:
train_df.loc[train_df['Age'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,NaN,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,NaN,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S
